## In this note, we conisder oringal [[18,4,4]] code with 32 physical qubits, including 18 data qubits and 14 check qubits, as we remove two extra Z check qubits and two extra X check qubits.
## We simultaneously prepare all check qubits in the initial step and simultaneously measure all check qubits in the final step of a full cycle of the syndrome measurement circuit. A full cycle of the SM circuit contain 9 layers.
## We conisder no reset is perfomed after each syndrome measurement cycle.

In [ ]:
import numpy as np
import random
from tabulate import tabulate
from scipy.sparse import coo_matrix, hstack 

from bposd.css import css_code
from ldpc import bposd_decoder, mod2
from mip import Model, xsum, minimize, BINARY, OptimizationStatus
import pickle

In [ ]:
from scipy.optimize import fsolve
import itertools
from itertools import combinations
from itertools import chain
from functools import reduce

from scipy.io import savemat
from scipy.io import loadmat
from tabulate import tabulate

# from exp_BB  import equations, new_sum, p_ij, correlation_nodes
from exp_ldpc import rank2, distance_test, simulate_circuitZ, simulate_circuitX, generate_noisy_circuit
# from exp_BB import revised_BB_code, SM_circuit, decoding_matrix_BB_code, num_end, simulate_Zerror_syndrome, simulate_Xerror_syndrome
from exp_BB import *

## Data from simulation of noisy SM circuits¶ 
(we do not reset check qubits during the experiment)

In [275]:
# number of Monte Carlo trials
num_trials = 100000
max_cycles = 4; 

# depolarizing noise model 
error_rate_init = 0.000
error_rate_idle = 0.000
error_rate_cnot = 0.003
error_rate_meas = 0.000

# no reset for check qubits after each SM cycle
cycle_no_reset = cycle[2*n2-len(rx)-len(rz):]

In [ ]:
Set_Zerror_syndrome_history = {} ;
Set_Zerror_syndrome_final_logical = {} ;

for num_cycles in range(1, max_cycles):

    # full syndrome measurement circuit
    # cycle_no_reset_repeated = num_cycles * cycle_no_reset

    Zerror_syndrome_history, Zerror_syndrome_final_logical =   \
        simulate_Zerror_syndrome(num_trials, num_cycles, cycle_no_reset, error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas) ;

    Set_Zerror_syndrome_history[num_cycles] = Zerror_syndrome_history
    Set_Zerror_syndrome_final_logical[num_cycles] = Zerror_syndrome_final_logical

In [276]:
Set_Xerror_syndrome_history = {} ;
Set_Xerror_syndrome_final_logical = {} ;

for num_cycles in range(3, max_cycles):

    # full syndrome measurement circuit
    # cycle_no_reset_repeated = num_cycles * cycle_no_reset

    Xerror_syndrome_history, Xerror_syndrome_final_logical =   \
        simulate_Xerror_syndrome(num_trials, num_cycles, cycle_no_reset, error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas) ;

    Set_Xerror_syndrome_history[num_cycles] = Xerror_syndrome_history
    Set_Xerror_syndrome_final_logical[num_cycles] = Xerror_syndrome_final_logical

In [387]:
# Set_Xerror_syndrome_history[2]

## Calculate the error probability
based on the expectation values of detections in syndrome_history

In [361]:
# Set_Xerror_syndrome_list = {} ;
Set_Xerror_syndrome_probability_list = {}  ;
Set_Zerror_syndrome_probability_list = {}  ;

for num_cycles in range(3, max_cycles):
    decoding_matrix = decoding_matrix_BB_code(k, nbs, cycle, num_cycles, lin_order,data_qubits,Zchecks,Xchecks,
                                              error_rate_init,error_rate_idle,error_rate_cnot,error_rate_meas, lz, lx  )
    
    Xerror_syndrome_list = [ _ for _ in decoding_matrix.HXdict] ;
    Xerror_syndrome_list = [ tuple(num for num in item if num < (n2-len(rz))*(num_cycles + num_end) ) for item in Xerror_syndrome_list]
    Xerror_syndrome_history = Set_Xerror_syndrome_history[num_cycles] ;
    
    temp1 = DetectionToSyndromeProbability(num_trials, num_cycles, Xerror_syndrome_list, Xerror_syndrome_history)
    Set_Xerror_syndrome_probability_list[num_cycles] = temp1.syndrome_probability_list ;
#-----------------------------------------------------------------------------------------------------------------------------
    # Zerror_syndrome_list = [ _ for _ in decoding_matrix.HZdict] ;
    # Zerror_syndrome_list = [ tuple(num for num in item if num < (n2-len(rx))*(num_cycles + num_end) ) for item in Zerror_syndrome_list]
    # Zerror_syndrome_history = Set_Zerror_syndrome_history[num_cycles] ;

    # temp2 = DetectionToSyndromeProbability(num_trials, num_cycles, Zerror_syndrome_list, Zerror_syndrome_history)
    # Set_Zerror_syndrome_probability_list[num_cycles] = temp2.syndrome_probability_list ;

Number of Z noisy circuits= 1032
Number of X noisy circuits= 1032
Number of distinct X-syndrome histories= 229
Number of distinct Z-syndrome histories= 229


In [ ]:
# Set_Xerror_syndrome_probability_list[1]
# list( Set_Xerror_syndrome_probability_list[1].values() )

In [380]:
Set_Xerror_syndrome_probability_list[3]

{(0, 7): 0.004646601591363986,
 (1, 8): 0.00489628120631308,
 (2, 9): 0.005176034708342103,
 (3, 10): 0.004767942718399032,
 (4, 11): 0.004279259645218081,
 (5, 12): 0.0053183693585751035,
 (6, 13): 0.005186176679303323,
 (0, 1, 4): 0.002931616452056507,
 (1, 2, 5): 0.0031691461432679087,
 (0, 2, 6): 0.000762595324415552,
 (0,): 0.0025570746104066943,
 (1, 3): 0.002961980529803663,
 (2, 3): 0.002924644576088902,
 (4, 5): 0.003142271977912046,
 (5, 6): 0.00330321829023726,
 (3, 4, 6): 0.0009563695461932742,
 (0, 2): 0.0008380612907016379,
 (0, 1): 0.0010517811820114154,
 (1, 2, 3): 0.0007375921990248781,
 (3, 4): 0.0007431423911324562,
 (5,): 0.0005521528105171716,
 (3, 6): 0.0008362271480511036,
 (0, 4, 6): 0.000681335166895862,
 (1, 4, 5): 0.000843855481935242,
 (2, 5, 6): 0.0007000875109388673,
 (4, 6, 7): 0.0014251781472684087,
 (4, 5, 8): 0.0017064633079134893,
 (5, 6, 9): 0.001443930491311414,
 (1, 2, 10): 0.0016064508063507938,
 (3, 11): 0.001518119891619736,
 (12,): 0.0037262784

In [385]:
# {k: v for k, v in Set_Xerror_syndrome_probability_list[3].items() if len(k) == 4}

## Draft

In [277]:
decoding_matrix = decoding_matrix_BB_code(k, nbs, cycle, num_cycles, lin_order,data_qubits,Zchecks,Xchecks,  \
                                          error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas, lz, lx )

syndrome_list = [ _ for _ in decoding_matrix.HXdict] ;
syndrome_list = [ tuple(num for num in item if num < (n2-len(rz))*(num_cycles + num_end) ) for item in syndrome_list]

detect_expectation = {} ;

for item in syndrome_list:
    temp_set = [tuple(comb) for r in range(2, len(item)+1) for comb in combinations(item, r) ] ;
    for key in temp_set:    
        detect_expectation[key] = None ;

for key in range( 7*(num_cycles + num_end) ):
    detect_expectation[(key,)] = None ;   

for s in detect_expectation:
    detect_expectation[tuple(s)] =  \
        np.sum([ np.prod([ Set_Xerror_syndrome_history[3][trial][k] for k in s ]) for trial in range(num_trials) ]) / num_trials

Number of Z noisy circuits= 1032
Number of X noisy circuits= 1032
Number of distinct X-syndrome histories= 229
Number of distinct Z-syndrome histories= 229


In [327]:
x_i = detect_expectation[(8,)] ;
x_j = detect_expectation[(13,)] ;
x_ij = detect_expectation[(8,13)] ;
result = p_ij(x_i, x_j, x_ij)
result

0.005072037041509214

In [386]:
solutions = [] 
deviation_list = []

ele = (8,11,13) 
# ele = (8+7,11+7,13+7) 
# ele = (8+14,11+14,13+14) 

x_1 = detect_expectation[(ele[0],)] ;
x_2 = detect_expectation[(ele[1],)] ;
x_12 = detect_expectation[(ele[0],ele[1])] ;

q12 = p_ij(x_1, x_2, x_12) ;
q1 = ((x_1) - q12)/(1 - 2*q12) ;
q2 = ((x_2) - q12)/(1 - 2*q12) ;

x_1 = detect_expectation[(ele[0],)] ;
x_3 = detect_expectation[(ele[2],)] ;
x_13 = detect_expectation[(ele[0],ele[2])] ;
r13 = p_ij(x_1, x_3, x_13) ;
r1 = ((x_1) - r13)/(1 - 2*r13) ;
r3 = ((x_3) - r13)/(1 - 2*r13) ;

x_2 = detect_expectation[(ele[1],)] ;
x_3 = detect_expectation[(ele[2],)] ;
x_23 = detect_expectation[(ele[1],ele[2])] ;
t23 = p_ij(x_2, x_3, x_23) ;
t2 = ((x_2) - t23)/(1 - 2*t23) ;

x_123 = detect_expectation[(ele[0],ele[1],ele[2])] ;

a123 = detect_expectation[(ele[0],ele[1],ele[2])] ;

for p123 in np.linspace(0.0001, 0.1, 10000):
    
    p12 = new_sub( q12, p123 )
    p13 = new_sub( new_sum(q1, q12), new_sum(r1, p123)   )
    p23 = new_sub( new_sum(q2, q12), new_sum(t2, p123)   )
    p1 = new_sub( new_sum(r1, p123),  q12  )
    p2 = new_sub( new_sum(t2, p123),  q12  )
    p3 = new_sub( r3, new_sub( new_sum(q2, q12), new_sum(t2, p123)   )   )

    temp4 = p123 * (1-p1) * (1-p2) * (1-p3) * (1-p12) * (1-p23) * (1-p13) \
       + p1 * p23 * (1-p123) * (1-p2) * (1-p3) * (1-p13) * (1-p12)   \
       + (1-p1) * p2 * p13 * (1-p123) * (1-p3) * (1-p23) * (1-p12)    \
       + (1-p1) * (1-p2) * p3 * p12 * (1-p123) * (1-p23) * (1-p13)   \
       + p123 * p1 * p23 * p2 * p13 * (1-p3) * (1-p12) \
       + p123 * p1 * p23 * p3 * p12 * (1-p2) * (1-p13) \
       + p123 * p2 * p13 * p3 * p12 * (1-p1) * (1-p23) \
       + (1 - p123) * p1 * p23 * p2 * p13 * p3 * p12 - x_123 ;
    
    if ( -1e-5 < temp4 < 1e-5 )  :
        # print(temp4)
        deviation_list.append(temp4)
        solutions.append(p123)

if len(solutions) == 0:
    print("Please try again, no solution.")

sorted_indices = np.argsort(np.abs(deviation_list)) 
solutions = [solutions[i] for i in sorted_indices ]

In [ ]:
"""
X_detection_list   X_detection_way    X_detect_expectation    X_detection_probability
syndrome_list   syndrome_four_ele_property   detect_expectation    syndrome_probability_list
"""

## Decoding error syndromes

In [ ]:
Set_bpdX = {} ;
Set_bpdZ = {} ;
Set_HZ = {}  ;
Set_HX = {}  ;
for num_cycles in range(1, max_cycles):
    
    decoding_matrix = decoding_matrix_BB_code(k, nbs, cycle, num_cycles, lin_order, data_qubits, Zchecks, Xchecks,
                                        error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas, lz, lx)

    Set_HZ[num_cycles] = decoding_matrix.HZ ;  Set_HX[num_cycles] = decoding_matrix.HX ;
    
    bpdX = bposd_decoder(
        decoding_matrix.HdecX,  #the parity check matrix
        channel_probs = list(Set_Xerror_syndrome_probability_list[num_cycles].values()) ,
        max_iter=my_max_iter, bp_method=my_bp_method, ms_scaling_factor=my_ms_scaling_factor, osd_method=my_osd_method, 
        osd_order=my_osd_order 
        )
    Set_bpdX[num_cycles] = bpdX ;

    bpdZ = bposd_decoder(
        decoding_matrix.HdecZ,  
        channel_probs = list(Set_Zerror_syndrome_probability_list[num_cycles].values()) ,
        max_iter=my_max_iter, bp_method=my_bp_method, ms_scaling_factor=my_ms_scaling_factor, osd_method=my_osd_method,  
        osd_order=my_osd_order 
        )
    Set_bpdZ[num_cycles] = bpdZ ;

In [ ]:
# correct Z-type error
Z_error_correction = {} ;
Z_no_error_correction = {} ;
for num_cycles in range(1, max_cycles):

    Z_good_trials = 0
    Z_good_list = [0, 0, 0, 0] ;
    
    Z_no_correction_good_list = [0, 0, 0, 0] ;
    Z_no_correction_good_trials = 0
    
    for trial in range(num_trials):
        syndrome_history = Set_Zerror_syndrome_history[num_cycles][trial]
        syndrome_final_logical = Set_Zerror_syndrome_final_logical[num_cycles][trial]
        
        Z_no_correction_good_list += ( syndrome_final_logical == np.array([0, 0, 0, 0])  )
        if np.array_equal( syndrome_final_logical, np.array([0, 0, 0, 0]) ):
            assert(  all(cc == 0 for cc in syndrome_final_logical )  )
            Z_no_correction_good_trials+=1

        bpdZ = Set_bpdZ[num_cycles] ;
        bpdZ.decode(syndrome_history)
        low_weight_error = bpdZ.osdw_decoding
        
        syndrome_history_augmented_guessed = (Set_HZ[num_cycles] @ low_weight_error) % 2
        syndrome_final_logical_guessed = syndrome_history_augmented_guessed[len(syndrome_history):(len(syndrome_history)+k)]  
        
        ec_resultZ = np.array_equal(syndrome_final_logical_guessed, syndrome_final_logical)
        correct_z = (syndrome_final_logical_guessed + syndrome_final_logical)   % 2
        
        Z_good_list += ( correct_z == np.array([0, 0, 0, 0])  )
    
        if ec_resultZ:
            assert(  all(cc == 0 for cc in correct_z )  )
            Z_good_trials+=1
    
    print(Z_no_correction_good_trials/num_trials)
    print(Z_good_trials/num_trials)
    
    Z_error_correction[num_cycles] = {"logical error propability": Z_good_trials/num_trials,
                                      "good_trials": Z_good_trials,
                                      "good_list": Z_good_list } ;
    
    
    Z_no_error_correction[num_cycles] = {"error propability": Z_no_correction_good_trials/num_trials,
                                         "good_trials": Z_no_correction_good_trials,
                                         "good_list": Z_no_correction_good_list   } ;

In [ ]:
# correct X-type error
X_error_correction = {} ;
X_no_error_correction = {} ;
for num_cycles in range(1, max_cycles):

    X_good_trials=0
    X_good_list = [0, 0, 0, 0] ;
    
    X_no_correction_good_list = [0, 0, 0, 0] ;
    X_no_correction_good_trials = 0
    
    for trial in range(num_trials):
        syndrome_history = Set_Xerror_syndrome_history[num_cycles][trial]
        syndrome_final_logical = Set_Xerror_syndrome_final_logical[num_cycles][trial]
        
        X_no_correction_good_list += ( syndrome_final_logical == np.array([0, 0, 0, 0])  )
        if np.array_equal( syndrome_final_logical, np.array([0, 0, 0, 0]) ):
            assert(  all(cc == 0 for cc in syndrome_final_logical )  )
            X_no_correction_good_trials+=1

        bpdX = Set_bpdX[num_cycles] ;
        bpdX.decode(syndrome_history)
        low_weight_error = bpdX.osdw_decoding
        
        syndrome_history_augmented_guessed = (Set_HX[num_cycles] @ low_weight_error) % 2
        syndrome_final_logical_guessed = syndrome_history_augmented_guessed[len(syndrome_history):(len(syndrome_history)+k)]  
        
        ec_resultX = np.array_equal(syndrome_final_logical_guessed, syndrome_final_logical)
        correct_x = (syndrome_final_logical_guessed + syndrome_final_logical)   % 2
        
        X_good_list += ( correct_x == np.array([0, 0, 0, 0])  )
    
        if ec_resultX:
            assert(  all(cc == 0 for cc in correct_x )  )
            X_good_trials+=1
    
    print(X_no_correction_good_trials/num_trials)
    print(X_good_trials/num_trials)
    
    X_error_correction[num_cycles] = {"logical error propability": X_good_trials/num_trials,
                                      "good_trials": X_good_trials,
                                      "good_list": X_good_list } ;
    
    
    X_no_error_correction[num_cycles] = {"error propability": X_no_correction_good_trials/num_trials,
                                         "good_trials": X_no_correction_good_trials,
                                         "good_list": X_no_correction_good_list   } ;    

## Save the data

In [ ]:
# savemat('Set_syndrome_history.mat', 
#         {  'Correct_X_trial_syndrome_history': Correct_X_trial_syndrome_history
#         })